In [15]:
import io
import numpy as np
import pandas as pd
import nibabel as nb
import trimesh
from PIL import Image
from sklearn import neighbors
import trimesh
import math
import open3d as o3d
from freesurfer_tools import transform_atlas

In [16]:
annot_paths = [
    "/home/mszul/git/DANC_multilayer_laminar/assets/lh.HCPMMP1.annot",
    "/home/mszul/git/DANC_multilayer_laminar/assets/rh.HCPMMP1.annot"
]
fsavg_sphere_paths = [
    "/home/mszul/git/DANC_multilayer_laminar/assets/lh.sphere.reg.gii",
    "/home/mszul/git/DANC_multilayer_laminar/assets/rh.sphere.reg.gii"
]
fsnat_sphere_paths = [
    "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/lh.sphere.reg.gii",
    "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/rh.sphere.reg.gii"
]
pial_path = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.gii"
pial_ds_path = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.ds.gii"
pial_ds_nodeep = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.ds.link_vector.nodeep.gii"
pial_ds_inflated = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.ds.inflated.nodeep.gii"
glasser = "/home/mszul/git/DANC_multilayer_laminar/assets/atlas_glasser_2016.csv"

In [17]:
atlas = pd.read_csv(glasser)

In [18]:
ml_deep = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.gii");
ml_nodeep = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.link_vector.nodeep.gii");

In [19]:
n_surf = 11

In [20]:
ml_nodeep = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.link_vector.nodeep.gii");
vertices, faces, normals = ml_nodeep.agg_data()
vertices = np.split(vertices, n_surf, axis=0)
faces = np.split(faces, n_surf, axis=0)
normals = np.split(normals, n_surf, axis=0)

In [22]:
mesh = trimesh.Trimesh(vertices=vertices[0], faces=faces[0], normals=normals[0], process=True, validate=True)
mesh = mesh.as_open3d
mesh.compute_vertex_normals(normalized=True)
# option.non_rigid_camera_coordinate = True
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True, point_show_normal=True)

In [24]:
mesh_colors, lab_col_map = transform_atlas(annot_paths, fsavg_sphere_paths, fsnat_sphere_paths, pial_path, pial_ds_path, pial_ds_nodeep)

In [25]:
mesh = trimesh.Trimesh(vertices=vertices[0], faces=faces[0], normals=normals[0], process=False, validate=False)
modified_mesh_colors = [np.append(i[:3], 255) for i in mesh_colors]
mesh = mesh.as_open3d
mesh.vertex_colors = o3d.utility.Vector3dVector(np.array([i[:3]/256 for i in mesh_colors]))
mesh.compute_vertex_normals(normalized=True)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
target_labels = ['L_MT_ROI']
ROI_maps = {i: [i == j.decode("utf=8") for j in lab_col_map] for i in target_labels}
n_surf = 11
mesh_colors = np.array([i[:3]/256 for i in mesh_colors])
layers_ROI = [trimesh.Trimesh(vertices=vertices[l], faces=faces[0], normals=normals[l], process=False, validate=False).as_open3d for l in range(n_surf)]
for ix, mesh in enumerate(layers_ROI):
    mesh.vertex_colors = o3d.utility.Vector3dVector(mesh_colors)
    mesh.remove_vertices_by_mask(~np.array(ROI_maps['L_MT_ROI']))
    mesh.compute_vertex_normals(normalized=True)
    mesh.paint_uniform_color(np.array([255,255,0])/255)
    mesh.translate(np.array([0, ix*2, ix*3]))
o3d.visualization.draw_geometries(layers_ROI, mesh_show_back_face=True)

[Open3D INFO]   -- Mouse view control --
[Open3D INFO]     Left button + drag         : Rotate.
[Open3D INFO]     Ctrl + left button + drag  : Translate.
[Open3D INFO]     Wheel button + drag        : Translate.
[Open3D INFO]     Shift + left button + drag : Roll.
[Open3D INFO]     Wheel                      : Zoom in/out.
[Open3D INFO] 
[Open3D INFO]   -- Keyboard view control --
[Open3D INFO]     [/]          : Increase/decrease field of view.
[Open3D INFO]     R            : Reset view point.
[Open3D INFO]     Ctrl/Cmd + C : Copy current view status into the clipboard.
[Open3D INFO]     Ctrl/Cmd + V : Paste view status from clipboard.
[Open3D INFO] 
[Open3D INFO]   -- General control --
[Open3D INFO]     Q, Esc       : Exit window.
[Open3D INFO]     H            : Print help message.
[Open3D INFO]     P, PrtScn    : Take a screen capture.
[Open3D INFO]     D            : Take a depth capture.
[Open3D INFO]     O            : Take a capture of current rendering settings.
[Open3D INFO]     Alt + Enter  : Toggle between full screen and windowed mode.
[Open3D INFO] 
[Open3D INFO]   -- Render mode control --
[Open3D INFO]     L            : Turn on/off lighting.
[Open3D INFO]     +/-          : Increase/decrease point size.
[Open3D INFO]     Ctrl + +/-   : Increase/decrease width of geometry::LineSet.
[Open3D INFO]     N            : Turn on/off point cloud normal rendering.
[Open3D INFO]     S            : Toggle between mesh flat shading and smooth shading.
[Open3D INFO]     W            : Turn on/off mesh wireframe.
[Open3D INFO]     B            : Turn on/off back face rendering.
[Open3D INFO]     I            : Turn on/off image zoom in interpolation.
[Open3D INFO]     T            : Toggle among image render:
[Open3D INFO]                    no stretch / keep ratio / freely stretch.
[Open3D INFO] 
[Open3D INFO]   -- Color control --
[Open3D INFO]     0..4,9       : Set point cloud color option.
[Open3D INFO]                    0 - Default behavior, render point color.
[Open3D INFO]                    1 - Render point color.
[Open3D INFO]                    2 - x coordinate as color.
[Open3D INFO]                    3 - y coordinate as color.
[Open3D INFO]                    4 - z coordinate as color.
[Open3D INFO]                    9 - normal as color.
[Open3D INFO]     Ctrl + 0..4,9: Set mesh color option.
[Open3D INFO]                    0 - Default behavior, render uniform gray color.
[Open3D INFO]                    1 - Render point color.
[Open3D INFO]                    2 - x coordinate as color.
[Open3D INFO]                    3 - y coordinate as color.
[Open3D INFO]                    4 - z coordinate as color.
[Open3D INFO]                    9 - normal as color.
[Open3D INFO]     Shift + 0..4 : Color map options.
[Open3D INFO]                    0 - Gray scale color.
[Open3D INFO]                    1 - JET color map.
[Open3D INFO]                    2 - SUMMER color map.
[Open3D INFO]                    3 - WINTER color map.
[Open3D INFO]                    4 - HOT color map.